# W&B의 Artifact
- 데이터, 모델, 또는 기타 파일을 체계적으로 저장하고 버전 관리할 수 있는 시스템
- Artifact는 W&B에서 데이터, 모델, 파일을 저장하고 체계적으로 추적하며, 재현성과 협업을 가능하게 하는 버전 관리 시스템


## 주요 특징
- 버전 관리
> 각 Artifact는 버전이 생성되므로 실험 간 비교와 재현성 확보할 수 있음
- 유형 지정
> 데이터(dataset), 모델(model), 분석 결과 등으로 유형을 지정할 수 있음
- 협업 및 공유
> 팀원들과 파일을 공유하거나 재사용할 수 있어 협업에 적합함


## 활용 예시
- 데이터셋 저장
> 데이터를 버전 관리하고 실험 간 동일한 데이터를 사용할 수 있게 함
- 모델 저장
> 모델의 체크포인트를 저장해 실험 결과를 재현 가능하게 함
- 학습 관리
> 모델 훈련에 필요한 종속 데이터셋이나 결과물을 체계적으로 관리

In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

DATA_PATH = "../data/"

SEED = 42 # 시드값

# 데이터 블러오기
train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터

# 결측치 처리
age_mean = train["age"].mean()
fare_median = train["fare"].median()
cabin_unk = "UNK"
embarked_mode = train["embarked"].mode()[0]
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

# 특성으로 사용할 변수 선택
cols = ["age","sibsp","parch","fare","pclass","gender","embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy()

# 범주형 변수 원핫인코딩
cols = ['gender','embarked']
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)


# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft = scaler.transform(train_ft)
test_ft = scaler.transform(test_ft)

# 정답 데이터
target = train["survived"].to_numpy().reshape(-1,1) # 정답 데이터 2차원으로 변경

class TitanicDataset(torch.utils.data.Dataset):
    def __init__(self, x, y=None):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        item = {}
        item["x"] = torch.Tensor(self.x[idx])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item

In [2]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class Net(torch.nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.seq = torch.nn.Sequential(
            torch.nn.Linear(n_features, 12),
            torch.nn.BatchNorm1d(12),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(12, 8),
            torch.nn.BatchNorm1d(8),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(8, 4),
            torch.nn.BatchNorm1d(4),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(4, 1)
        )
    def forward(self, x):
        return self.seq(x)

def train_loop(dl, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in dl:
        pred = model(batch["x"].to(device))
        loss = loss_fn(pred, batch["y"].to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dl)
    return epoch_loss

@torch.no_grad()
def test_loop(dl, model, loss_fn, device):
    epoch_loss = 0
    model.eval()

    act = torch.nn.Sigmoid()
    pred_list = []
    for batch in dl:
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn(pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act(pred)
        pred = pred.to("cpu").numpy()
        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dl)
    return epoch_loss, pred

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

loss_fn = torch.nn.BCEWithLogitsLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
n_features = train_ft.shape[1]
n_splits = 5

cv = KFold(n_splits, shuffle=True, random_state=SEED)

In [3]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: koeyhi (koeyhi-student). Use `wandb login --relogin` to force relogin


True

In [38]:
init_config = {
    "optimizer": "adam",
    "batch_size": batch_size,
    "lr": 0.001,
    "validation": f"{n_splits}-fold",
    "n_features": n_features,
}

run = wandb.init(
    project="my-project",
    config=init_config
)

0-fold-auc,▁██▇▇██▁██▇▇██
0-fold-train_loss,█▆▄▃▂▁▁█▆▄▃▂▁▁
0-fold-valid_loss,█▅▃▂▂▁▁█▅▃▂▂▁▁
1-fold-auc,▁▆▇██████████
1-fold-train_loss,█▆▅▅▄▃▃▂▂▁▁▁▁
1-fold-valid_loss,█▅▅▄▃▃▂▂▂▂▁▁▁
2-fold-auc,▁▅▇████████████████
2-fold-train_loss,█▆▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
2-fold-valid_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
3-fold-auc,▁▄▆▆▇▇▇▇▇▇▇▇█████████████████
3-fold-train_loss,█▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁


# wandb.Artifact
- WandB에서 버전 관리 대상이 되는 파일들을 담을 수 있는 artifact 객체 반환
- artifact 객체의 메서드를 이용해서 버전 관리 대상이 되는 파일들을 담고, run 객체의 log_artifact 메서드에 전달
- 주요파라미터
    - name: 아티펙트 이름
    - type: 아티펙트 유형
    - description: 아티펙트 설명

In [39]:
artifact = wandb.Artifact(name="titanic-dataset", type="dataset")

## artifact 객체의 `add_dir`, `add_file` 메서드
- 폴더 또는 파일을 아티펙트에 추가
- 주요 파라미터
    - local_path: 저장할 폴더 및 파일의 경로 지정
    - name: 아티펙트 내에서 사용될 폴더 및 파일 경로

In [40]:
np.save("train_ft.npy", train_ft)
np.save("target.npy", target)
np.save("test_ft.npy", test_ft)

In [41]:
artifact.add_file("train_ft.npy", name="train/train_ft.npy")
artifact.add_file("target.npy", name="train/target.npy")
artifact.add_file("test_ft.npy", name="test/test_ft.npy")

ArtifactManifestEntry(path='test/test_ft.npy', digest='GGdqpwHXS0klVG7hiEwMAA==', size=31568, local_path='/root/.local/share/wandb/artifacts/staging/tmpq491_zq8', skip_cache=False)

In [42]:
run.log_artifact(artifact)

<Artifact titanic-dataset>

In [43]:
reset_seeds(SEED)
score_list = []
artifact = wandb.Artifact(name="titanic-model", type="model")

for i, (tri, vai) in enumerate(cv.split(train_ft)):
    train_dt = TitanicDataset(train_ft[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    valid_dt = TitanicDataset(train_ft[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    model = Net(n_features).to(device)
    optimizer = torch.optim.Adam(model.parameters())

    patience = 0
    best_score = 0


    for _ in range(100):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)
        score = roc_auc_score(target[vai], pred)
        run.log({
            f"{i}-fold-auc": score,
            f"{i}-fold-train_loss": train_loss,
            f"{i}-fold-valid_loss": valid_loss,
        })

        patience += 1
        if score > best_score:
            best_score = score
            patience = 0
            torch.save(model.state_dict(), f"model_{i}.pth")

        if patience == 5:
            break

    run.summary[f"{i}-fold-auc"] = best_score

    score_list.append(best_score)

    artifact.add_file(f"model_{i}.pth", f"model_{i}.pth")

In [44]:
run.summary["cv-score"] = np.mean(score_list)

In [45]:
run.log_artifact(artifact)

<Artifact titanic-model>

In [46]:
run.finish()

0-fold-auc,▁██▇▇██
0-fold-train_loss,█▆▄▃▂▁▁
0-fold-valid_loss,█▅▃▂▂▁▁
1-fold-auc,▁▆▇██████████
1-fold-train_loss,█▆▅▅▄▃▃▂▂▁▁▁▁
1-fold-valid_loss,█▅▅▄▃▃▂▂▂▂▁▁▁
2-fold-auc,▁▅▇████████████████
2-fold-train_loss,█▆▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
2-fold-valid_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
3-fold-auc,▁▄▆▆▇▇▇▇▇▇▇▇█████████████████
3-fold-train_loss,█▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁


# 예측하기
1. run 객체 생성
2. run 객체의 `use_artifact` 메서드에 artifact 경로 전달
    - artifact 객체 반환
    - artifact 경로
        - `<팀명>/<프로젝트명>/<아티펙트명>:<version>`
        - 버전 예시: v0, v1, latest
3. artifact 객체의 download 메서드 실행

In [47]:
run = wandb.init(project="my-project", name="inference")

In [52]:
artifact = run.use_artifact("my-project/titanic-model:latest", type="model")

In [51]:
artifact.download("models")

wandb:   5 of 5 files downloaded.  


'models'

In [53]:
artifact = run.use_artifact("my-project/titanic-dataset:latest", type="dataset")
artifact.download("data")

wandb:   3 of 3 files downloaded.  


'data'

In [54]:
test_dt = TitanicDataset(test_ft)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size, False)

In [55]:
pred_list = []
for i in range(n_splits):
    model = Net(n_features).to(device)
    state_dict = torch.load(f"models/model_{i}.pth", weights_only=True)
    model.load_state_dict(state_dict)
    _, pred = test_loop(test_dl, model, loss_fn, device)
    pred_list.append(pred)

In [61]:
pred = (np.mean(pred_list, axis=0) > 0.5).astype(int)
pred.shape

(393, 1)

# wandb.Table
- 테이블 형식으로 데이터 로깅
- 주요 파라미터
    - columns: 컬럼명 리스트 전달
    - data: 데이터 리스트 전달
    - dataframe: 데이터프레임 전달

In [67]:
pred_arr = np.concatenate([test["passengerid"].to_numpy().reshape(-1, 1), pred], axis=1)
pred_table = wandb.Table(["test_id", "prediction"], pred_arr)

In [68]:
pred_df = pd.DataFrame(pred_arr, columns=["test_id", "prediction"])
pred_table = wandb.Table(dataframe=pred_df)

In [69]:
run.log({"test predictions": pred_table})

In [70]:
run.finish()